In [ ]:
'''
https://github.com/breezedeus/pix2text
https://pix2text.readthedocs.io/zh-cn/stable/examples_en/

it is complicated to understand its code. Meanwhile, the generated image
is not accurate, which might be caused by the refined model of yolo7-mini because
it uses refined yolo7-mini to cut out the page and identify different blocks.

- the generated latex, formula part latex is always repeating twice. Need to investigate
- lack of the vivid way to mark the boxes of each idenfied blocks and a way to
rebuild the image by sequencing the ouput blocks.

'''

'\nhttps://github.com/breezedeus/pix2text\nhttps://pix2text.readthedocs.io/zh-cn/stable/examples_en/\n'

# Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = '/content/drive/MyDrive/claudiaYao - Colab Notebooks/PDF_SUMMARY'

pdf_path = cur_path + "/pdf"
data_path = cur_path + "/data"
output_path = cur_path + "/output"

if not os.path.exists(data_path):
    os.mkdir(data_path)

In [ ]:
!pip install pix2text

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [ ]:

p2t = Pix2Text.from_config()




Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /usr/local/lib/python3.10/dist-packages/pix2text/doc_xl_layout/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}


In [ ]:
from pix2text import set_logger, Pix2Text
from pix2text.layout_parser import ElementType
from PIL import Image
import string
import fitz
import sys
import os
import glob


# get one page image and return the output md file and all the titles, figures, tables on the page.
def get_page_content(page_img, page_num):
    titles = []
    figures = []
    page_output_path = output_path +'/' + str(page_num) + '/output-md'
    try:
        doc = p2t.recognize_page(page_img, table_as_image=True, text_contain_formula=False, save_debug_res=output_path + "/log") #, page_numbers=[0, 1])
        doc.to_markdown(page_output_path)  # The exported Markdown information is saved in the output-md directory

        for i, _ in enumerate(doc.elements):
            # pick out the title so that we could set up the questions for RAG
            if doc.elements[i].type == ElementType.TITLE:
                print("title:", doc.elements[i].text)
                titles.append(doc.elements[i].text)

    except Exception as e:
        print(e)

    return page_output_path, titles



def get_pdf_content(pdf_file):
    all_titles = []

    try:
        fitz.TOOLS.mupdf_warnings()  # empty the problem message container
        doc = fitz.open(pdf_file)
        warnings = fitz.TOOLS.mupdf_warnings()
        if warnings:
            print(warnings)
            raise RuntimeError()

        for i, page in enumerate(doc):  # iterate through the pages
            try:
                img = page.get_pixmap()  # render page to an image
                img_path = output_path + f"/{i}.png"
                img.save(img_path)  # store image as a PNG
                page_folder, titles = get_page_content(img_path, i)

                # append all the text
                with open(page_folder + "/output.md", "r") as newfile:
                    text = newfile.read()


                with open(output_path + "/all_text.md", "a") as myfile:
                    myfile.write(text + "\n")

                if titles:
                    all_titles.extend(titles)
            except:
                print("error when handling the image file {} -page {}".format(pdf_file, i))
                return None

        return all_titles

    except:
        print("error when opening the pdf file {}".format(pdf_file))
        return None

def parse_titles(title_list):
    caption = title_list[0]
    authors = title_list[1]
    sections ={}
    section_titles = ['I.', 'II.', 'III.', 'IV.', 'V.', 'VI.', 'VII.', 'VIII.', 'IX.', 'X']
    subsection_titles = ['A.', 'B.', 'C.', 'D.', 'E.', 'F.', 'G.', 'H.', 'I.']
    cur_title =''

    for title in title_list[2:]:
        t = title.split()[0].strip()
        if t in section_titles:
            sections[title] = []
            cur_title = title
        elif t in subsection_titles:
            sections[cur_title].append(title)
    return sections



## Convert PDF file into MD file, together with figures, tables, and title list
- the figures and tables are located in folder output/{page_number}/output-md/figures/
- the page md file are located in folder output/{page_number}/output-md/output.md
- all the log files for all the pages are located in folder output/log/

In [ ]:


all_titles = get_pdf_content("/content/drive/MyDrive/claudiaYao - Colab Notebooks/PDF_SUMMARY/pdf/1901.00031v1.pdf")

display(parse_titles(all_titles))



WARNING ⚠️ imgsz=[993, 768] must be multiple of max stride 32, updating to [1024, 768]
0: 1024x768 (no detections), 1567.7ms
Speed: 9.6ms preprocess, 1567.7ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 768)


0it [00:00, ?it/s]


title: Interest Point Detection based on Adaptive Ternary Coding
title: Zhenwei Miao, Membe, IEEE, Kim-Hui Yap, Senior Member, IEEE and Xudong Jiang, Senior Member, IEEE
title: I. INTRODUC'TION
title: II. THE PROPOSED INTEREST POINT DETECTOR
title: A. Prnblem Formulavion
/content/drive/MyDrive/claudiaYao - Colab Notebooks/PDF_SUMMARY/output/0/output-md


100%|██████████| 1/1 [00:12<00:00, 12.07s/it]



WARNING ⚠️ imgsz=[993, 768] must be multiple of max stride 32, updating to [1024, 768]
0: 1024x768 (no detections), 1066.3ms
Speed: 6.3ms preprocess, 1066.3ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 768)


...

0it [00:00, ?it/s]


title: IV. CONCLUSIONS
/content/drive/MyDrive/claudiaYao - Colab Notebooks/PDF_SUMMARY/output/4/output-md

WARNING ⚠️ imgsz=[993, 768] must be multiple of max stride 32, updating to [1024, 768]
0: 1024x768 (no detections), 1076.9ms
Speed: 6.5ms preprocess, 1076.9ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 768)


0it [00:00, ?it/s]


/content/drive/MyDrive/claudiaYao - Colab Notebooks/PDF_SUMMARY/output/5/output-md


[]

In [ ]:
# after getting all the titles and whole text md, we could set up question set to build RAG

display(parse_titles(all_titles))

{"I. INTRODUC'TION": [],
 'II. THE PROPOSED INTEREST POINT DETECTOR': ['A. Prnblem Formulavion',
  'B. Adapvive Ternary Coding Algovitum',
  'C. The Pnpsed ATC Deteetor'],
 'III. EXPERIMENTS': ['B. Applicanion to Face Reeognition'],
 'IV. CONCLUSIONS': []}